In [1]:
! nvidia-smi

Tue Nov 28 09:19:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.33       Driver Version: 528.33       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   64C    P0    16W /  60W |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
! pip install transformers datasets accelerate peft

  Using cached multidict-6.0.4-cp39-cp39-win_amd64.whl (28 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-win_amd64.whl.metadata (5.3 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/521.2 kB ? eta -:--:--
   --- ----------------------------------- 41.0/521.2 kB 960.0 kB/s eta 0:00:01
   ---- ---------------------------------- 61.4/521.2 kB 648.1 kB/s eta 0:00:01
   -------- ----------------------------- 122.9/521.2 kB 901.1 kB/s eta 0:00:01
   ------------- -------------------------- 174.1/521.2 kB 1.0 MB/s eta 0:00:01
   ------------------ --------------------- 245.8/521.2 kB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 286.7/521.2 kB 1.0 MB/s eta 0:00:01
   ---------------------------- ----------- 368.6/521.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------- ----- 450.6/521.2 kB 1.2 MB/s eta 0:00:01
   ----------------------

In [3]:
! pip install transformers datasets

In [4]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

c:\Users\valid\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_df = pd.read_parquet("train.parquet")
test_df = pd.read_parquet("test.parquet")
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [6]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 849kB/s]
c:\Users\valid\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\valid\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 1.11MB/s

In [ ]:
def preprocess_function(sample,padding="max_length"):
    model_inputs = tokenizer(sample["Human"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["Assistant"], max_length=256, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.1,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4718592 || all params: 787868672 || trainable%: 0.5989059049678777


In [ ]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
output_dir="lora-flan-t5-large-chat"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
)
model.config.use_cache = False

trainer.train()
peft_save_model_id="lora-flan-t5-large-chat"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)

Cloning https://huggingface.co/Vasanth/lora-flan-t5-large-chat into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
3750,2.031200


pytorch_model.bin:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

In [ ]:
! cp -r /content/lora-flan-t5-large-chat/ /content/drive/MyDrive/Chatbot/

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "lora-flan-t5-large-chat"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0}).cuda()
model.eval()

sample = "Human: \nExplain me about the working of Artificial Intelligence. \nAssistant: "
input_ids = tokenizer(sample, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
print(f"{sample}")

print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])


Human: 
Explain me about the working of Artificial Intelligence. 
Assistant: 
Artificial intelligence, also known as AI, is the process of developing artificial intelligence algorithms for computer programs, such as robotics or machine learning. These algorithms typically employ computer algorithms to understand and solve problems that are more complex than that of human beings. These algorithms are trained to interpret information, identify patterns, and predict outcomes. They are able to make calculations, solve problems, and detect anomalies. They are also trained to think in terms of patterns and to recognize and understand human emotions, thoughts, and emotions. They are trained to understand different kinds of contexts and context-specific information. The algorithms used in AI have many advantages over humans in many ways: - Adaptable: AI algorithms can adapt to changing business conditions, technological changes, and changing technological conditions. This can improve the effec